# Draft analysis 

---

Group name: E 

---


## Introduction

Durch die Digitalisierung der Versicherungsbranche wurde eine neue Norm gesetzt, die es ermöglichte besser auf die Bedürfnisse der Kunden einzugehen. Die Versicherungen stellten sich damit einer bereits bekannten Herausforderung, dem Versicherungsbetrug. Jede zehnte Schadensmeldung ist betrugsverdächtig und kostet den deutschen Versicherungen rund fünf Milliarden Euro pro Jahr (GDV, 2020). Um Versicherungsbetrug zu erkennen, wird spezielle Betrugserkennungssoftware verwendet. Hierbei nutzen Versicherungsbetrüger zunehmend Anleitungen aus dem Internet, die es erleichtern die Betrugserkennungssoftware zu umgehen (GDV, 2020). 

*Quelle: https://www.gdv.de/gdv/medien/medieninformationen/sorge-der-versicherer-corona-gibt-betruegern-auftrieb-61842*

Mit diesem Datensatz untersuchen wir, ob und welche Maßnahmen nötig sind, um den Prozess der Erkennung von Versicherungsbetrug zu verbessern. 

Wie können Betrugsfälle von der Betrugserkennungssoftware und Mitarbeiter besser erkannt werden? Welche Maßnahmen sind dazu nötig?

Wir haben folgende Hypothesen:

1. Unter 35 jährige Versicherungskunden betrügen öfters.

2. Männer betrügen häufiger als Frauen.

3. Personen mit einem niedrigem Bildungsabschluss betrügen häufiger.

4. Schadensfälle ohne polizeilicher Dokumentation sind häufiger Betrugsfälle.

5. Schadensfälle mit mehreren beteiligten Fahrzeugen sind weniger in Betrugsfälle verwickelt.

## Setup

In [1]:
import pandas as pd
import altair as alt
import numpy as np

## Data

## Import data

In [2]:
ROOT= "https://raw.githubusercontent.com/christophersegatz/dst-project/main/data/insurance_fraud/"
DATA = "fraud.csv"

df=pd.read_csv(ROOT + DATA)


### Data structure

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28836 entries, 0 to 28835
Data columns (total 38 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   CustomerID                  28836 non-null  object 
 1   DateOfIncident              28836 non-null  object 
 2   TypeOfIncident              28836 non-null  object 
 3   TypeOfCollission            28836 non-null  object 
 4   SeverityOfIncident          28836 non-null  object 
 5   AuthoritiesContacted        28836 non-null  object 
 6   IncidentState               28836 non-null  object 
 7   IncidentCity                28836 non-null  object 
 8   IncidentAddress             28836 non-null  object 
 9   IncidentTime                28836 non-null  int64  
 10  NumberOfVehicles            28836 non-null  int64  
 11  PropertyDamage              28836 non-null  object 
 12  BodilyInjuries              28836 non-null  int64  
 13  Witnesses                   288

### Data corrections

In [4]:
df.drop(["DateOfIncident","TypeOfCollission", "SeverityOfIncident", "IncidentState", "IncidentCity", "IncidentAddress", "IncidentTime",
         "NumberOfVehicles", "PropertyDamage", "BodilyInjuries", "Witnesses", "PoliceReport", "AmountOfTotalClaim", "AmountOfInjuryClaim",
        "AmountOfPropertyClaim", "AmountOfVehicleDamage", "InsuredZipCode", "InsuredOccupation", "InsuredHobbies",
        "CapitalGains", "CapitalLoss", "Country", "InsurancePolicyNumber", "DateOfPolicyCoverage", "InsurancePolicyState", "Policy_CombinedSingleLimit",
        "Policy_Deductible", "PolicyAnnualPremium", "UmbrellaLimit", "InsuredRelationship"], axis= 1, inplace=True)

In [5]:
df = df.astype(
    {
        "TypeOfIncident": "category",
        "AuthoritiesContacted": "category",
        "InsuredGender": "category",
        "InsuredEducationLevel": "category"

})

## Analysis

### Descriptive statistics

In [6]:
# make a list of variables you want to use
var_list = ['TypeOfIncident']

# create a new dataframe called source with only var_list
source = df[var_list]

source['TypeOfIncident'].cat.categories
source['TypeOfIncident'].value_counts()

Multi-vehicle Collision     11966
Single Vehicle Collision    11677
Vehicle Theft                2685
Parked Car                   2508
Name: TypeOfIncident, dtype: int64

In [7]:
# make a list of variables you want to use
var_list = ['AuthoritiesContacted']

# create a new dataframe called source with only var_list
source = df[var_list]

source['AuthoritiesContacted'].cat.categories
source['AuthoritiesContacted'].value_counts()

Police       8324
Fire         6518
Ambulance    5732
Other        5570
None         2692
Name: AuthoritiesContacted, dtype: int64

In [8]:
alt.data_transformers.disable_max_rows()

# make a list of variables you want to use
var_list = ['InsuredAge']

# create a new dataframe called source with only var_list
source = df[var_list]

alt.Chart(source).mark_bar().encode(
    alt.X("InsuredAge", bin=True),
    y='count()',
)

/Users/raphael/opt/anaconda3/envs/dst/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)

In [9]:
# make a list of variables you want to use
var_list = ['InsuredGender']

# create a new dataframe called source with only var_list
source = df[var_list]

source['InsuredGender'].cat.categories
source['InsuredGender'].value_counts()

FEMALE    15644
MALE      13162
Name: InsuredGender, dtype: int64

In [10]:
# make a list of variables you want to use
var_list = ['InsuredEducationLevel']

# create a new dataframe called source with only var_list
source = df[var_list]

source['InsuredEducationLevel'].cat.categories
source['InsuredEducationLevel'].value_counts()

JD             4808
High School    4583
MD             4161
Masters        4141
Associate      4125
PhD            3556
College        3462
Name: InsuredEducationLevel, dtype: int64

### Exploratory data analysis

H1: Unter 35 jährige Versicherungskunden betrügen öfters.

In [73]:

df['is_over_35'] = np.where(df['InsuredAge']>=35, 'yes', 'no')

pd.crosstab(index=df['is_over_35'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

ReportedFraud,N,Y
is_over_35,,
no,72.4,27.6
yes,73.3,26.7


Kunden unter 35 Jahren Betrügen häufiger allerdings nur um 0.9 Prozentpunkte

H2: Männer betrügen häufiger als Frauen.

In [12]:
pd.crosstab(index=df['InsuredGender'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

ReportedFraud,N,Y
InsuredGender,,
FEMALE,74.03,25.97
MALE,71.75,28.25


Bei den Männern betrügen 28.25% - bei den Frauen 25.97%.

H3: Personen mit einem niedrigem Bildungsabschluss betrügen häufiger.

In [13]:
pd.crosstab(index=df['InsuredEducationLevel'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

ReportedFraud,N,Y
InsuredEducationLevel,,
Associate,74.45,25.55
College,70.45,29.55
High School,73.99,26.01
JD,74.29,25.71
MD,72.27,27.73
Masters,74.55,25.45
PhD,69.85,30.15


Der höchste Bildungsabschluss -PhD hat mit 30.15%. Der niedrigste Bildungsabschlus - Associate hat mit 25.55% die zweitniedrigste Betrugsrate.

H4: Schadensfälle ohne polizeilicher Dokumentation sind häufiger Betrugsfälle.

In [14]:
pd.crosstab(index=df['AuthoritiesContacted'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

ReportedFraud,N,Y
AuthoritiesContacted,,
Ambulance,69.28,30.72
Fire,70.60,29.40
None,90.12,9.88
Other,67.58,32.42
Police,75.54,24.46


Wurder keine Authority kontaktiert ist die Betrugsrtate mit 9.88% am geringsten. Wurde die Polizei kontaktiert ist die Betrugsrate mit 24.46% am 2. besten

H5: Schadensfälle mit mehreren beteiligten Fahrzeugen sind weniger in Betrugsfälle verwickelt.

In [15]:
pd.crosstab(index=df['TypeOfIncident'], 
            columns=df['ReportedFraud'],
            normalize='index').round(4) * 100

ReportedFraud,N,Y
TypeOfIncident,,
Multi-vehicle Collision,70.75,29.25
Parked Car,87.88,12.12
Single Vehicle Collision,68.96,31.04
Vehicle Theft,86.74,13.26


Die Hypothese kann nicht angenommen werden. Unfälle aus der Kategorie Parked Car und Vehicle Theft haben die geringste Betrugsrate. 

## Visualizations

### Visualization ideas

### Save Visualizations



Save your draft visualizations in the folder `reports/visualizations/`. Use a meaningful name (always include the word `draft` and a `timestamp`in your filename).

## Conclusion and recommended action